In [2]:
dir="/home/davide/Downloads/fimo"
genbank_dir = "/home/davide/Desktop/genomiChro/genbanks_prokka"
import os
import pandas as pd
from Bio import SeqIO
import re
#open all tsv files in the directory
#create new dataframe
def get_intergenic_length(fimo_file, locus_tag):
    intergen_dir = "/home/davide/Desktop/genomiChro/intergeniche_tutte"
    #open fasta file with bio python and check the length of the intergenic sequence using len(record.seq)
    for fasta in os.listdir(intergen_dir):
        if fimo_file[4:-10] in fasta:
            for record in SeqIO.parse(os.path.join(intergen_dir,fasta), 'fasta'):
                if locus_tag in record.description:
                    return len(record.seq)

dataframe= pd.DataFrame(columns=['gene'])
for file in os.listdir(dir):
    #skip hidden files
    if file.startswith("."):
        continue
    data = pd.read_csv(os.path.join(dir, file), delimiter='\t', index_col=None)
    #delete last 4 rows
    data = data[:-4]
    #reorder columns with seqeunce_name first
    data = data[['sequence_name', 'start','matched_sequence']]
    data['start'] = data['start'].astype(int)
    lunghezze_intergeniche = data['sequence_name'].apply(lambda x: get_intergenic_length(file,x))
    data['start'] =  data['start'] - lunghezze_intergeniche
    #data['matched_sequence'] = data['matched_sequence'] + "(" + data['start'].astype(str) + ")"
    #data = data.drop(columns=['start'])
    #print(data.shape)
    for file2 in os.listdir(genbank_dir):
        if file[4:-10] in file2:
            genoma=SeqIO.parse(os.path.join(genbank_dir, file2), 'genbank')
            for seq in genoma:
                for record in seq.features:
                    if record.type=='CDS' and record.qualifiers['locus_tag'][0] in data['sequence_name'].values:
                        if record.qualifiers.get('gene',0):
                            gene=record.qualifiers['gene'][0]
                            locus_tag=record.qualifiers['locus_tag'][0]
                            data['sequence_name'] = data['sequence_name'].replace(record.qualifiers['locus_tag'][0], gene)
                        else:
                            #delete the row that contains the sequence name
                            data = data[data.sequence_name != record.qualifiers['locus_tag'][0]]
    data['matched_sequence'] = data['matched_sequence'] + "(" + data['start'].astype(str) + ")"
    data = data.drop(columns=['start'])
    data.rename(columns={'sequence_name':'gene'}, inplace=True)
    data.rename(columns={'matched_sequence': file[4:-12]}, inplace=True)
    dataframe = pd.merge(dataframe, data, on='gene', how='outer')
#reorder rows sort for number of Nan values
dataframe = dataframe.reindex(dataframe.isnull().sum(axis=1).sort_values().index)

In [3]:
dataframe.info

<bound method DataFrame.info of        gene           GCF_014696895           GCF_000317125  \
130    phrA   TAGTACGAGTGAACTA(-29)   TAGTACGAGTGAACTA(-29)   
145    recA   TAGTATATCTGCACTA(-66)   TAGTATATCTGCACTA(-66)   
176   smc_1   CAGTGCATCTGTACTG(-51)   CAGTGCATCTGTACTG(-51)   
203     xfp   AAGTACAAATACTCAA(-34)   AAGTACAAATACTCAA(-34)   
189    trpF  TAGTAAAATTGTATTA(-143)  TAGTAAAATTGTATTA(-144)   
..      ...                     ...                     ...   
88   htpX_5  GAGTAGATCCGTACTC(-135)                     NaN   
90     ipsA                     NaN                     NaN   
91   iscS_1                     NaN                     NaN   
80   hisC_5                     NaN                     NaN   
218  zosA_3                     NaN                     NaN   

              GCF_003015105           GCF_032860985           GCF_003991895  \
130   TAGTACGAGTGAACTA(-29)   TAGTACGAGTGAACTA(-29)   TAGTACGAGTGAACTA(-29)   
145   TAGTATATCTGCACTA(-66)   TAGTATATCTGCACTA(-66)  

In [4]:
dataframe.head()

,gene,GCF_014696895,GCF_000317125,GCF_003015105,GCF_032860985,GCF_003991895,GCF_002939305,GCF_034438355,GCF_003003835,CCM,GCF_023546805
130,phrA,TAGTACGAGTGAACTA(-29),TAGTACGAGTGAACTA(-29),TAGTACGAGTGAACTA(-29),TAGTACGAGTGAACTA(-29),TAGTACGAGTGAACTA(-29),TAGTACAAAAGAACTA(-30),TAGTACGAGTGAACTA(-29),TAGTACGAGTGAACTA(-29),TAGTACATTGTTACTA(-40),TAGTACGAGTGAACTA(-29)
145,recA,TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-62),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66)
176,smc_1,CAGTGCATCTGTACTG(-51),CAGTGCATCTGTACTG(-51),CAGTGCATCTGTACTG(-51),CAGTGCATCTGTACTG(-51),CAGTGCATCTGTACTG(-51),NaN,CAGTGCATCTGTACTG(-51),TAGTACGGTTGTTTTT(-21),CAGTGCAAATGTACTG(-60),CAGTGCATCTGTACTG(-51)
203,xfp,AAGTACAAATACTCAA(-34),AAGTACAAATACTCAA(-34),AAGTACAAATACTCAA(-34),AAGTACAAATACTCAA(-34),AAGTACAAATACTCAA(-34),NaN,AAGTACAAATACTCAA(-34),AAGTACAAATACTCAA(-34),NaN,AAGTACAAATACTCAA(-34)
189,trpF,TAGTAAAATTGTATTA(-143),TAGTAAAATTGTATTA(-144),TAGTAAAATTGTATTA(-144),TAGTAAAATTGTATTA(-144),TAGTAAAATTGTATTA(-144),NaN,TAGTAAAATTGTATTA(-144),TAGTAAAATTGTATTA(-144),NaN,TAGTAAAATTGTATTA(-144)


In [5]:
dataframe

,gene,GCF_014696895,GCF_000317125,GCF_003015105,GCF_032860985,GCF_003991895,GCF_002939305,GCF_034438355,GCF_003003835,CCM,GCF_023546805
130,phrA,TAGTACGAGTGAACTA(-29),TAGTACGAGTGAACTA(-29),TAGTACGAGTGAACTA(-29),TAGTACGAGTGAACTA(-29),TAGTACGAGTGAACTA(-29),TAGTACAAAAGAACTA(-30),TAGTACGAGTGAACTA(-29),TAGTACGAGTGAACTA(-29),TAGTACATTGTTACTA(-40),TAGTACGAGTGAACTA(-29)
145,recA,TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-62),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66),TAGTATATCTGCACTA(-66)
176,smc_1,CAGTGCATCTGTACTG(-51),CAGTGCATCTGTACTG(-51),CAGTGCATCTGTACTG(-51),CAGTGCATCTGTACTG(-51),CAGTGCATCTGTACTG(-51),NaN,CAGTGCATCTGTACTG(-51),TAGTACGGTTGTTTTT(-21),CAGTGCAAATGTACTG(-60),CAGTGCATCTGTACTG(-51)
203,xfp,AAGTACAAATACTCAA(-34),AAGTACAAATACTCAA(-34),AAGTACAAATACTCAA(-34),AAGTACAAATACTCAA(-34),AAGTACAAATACTCAA(-34),NaN,AAGTACAAATACTCAA(-34),AAGTACAAATACTCAA(-34),NaN,AAGTACAAATACTCAA(-34)
189,trpF,TAGTAAAATTGTATTA(-143),TAGTAAAATTGTATTA(-144),TAGTAAAATTGTATTA(-144),TAGTAAAATTGTATTA(-144),TAGTAAAATTGTATTA(-144),NaN,TAGTAAAATTGTATTA(-144),TAGTAAAATTGTATTA(-144),NaN,TAGTAAAATTGTATTA(-144)
...,...,...,...,...,...,...,...,...,...,...,...
88,htpX_5,GAGTAGATCCGTACTC(-135),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,ipsA,NaN,NaN,NaN,NaN,NaN,TAGTTCATATATACTC(-141),NaN,NaN,NaN,NaN
91,iscS_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAGTAAACTTGTACTA(-223),NaN
80,hisC_5,NaN,NaN,TAGTTCAAATACACTG(-124),NaN,NaN,NaN,NaN,NaN,NaN,NaN
